In [62]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression

In [3]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 000D-0704

 Directory of C:\Users\lv.shuo\Documents\GitHub\Kaggle\Whats cooking

2015/11/13  16:59    <DIR>          .
2015/11/13  16:59    <DIR>          ..
2015/11/13  16:59    <DIR>          .ipynb_checkpoints
2015/11/13  16:58                78 ingredients.ipynb
2015/08/26  15:11           137,045 sample_submission.csv
2015/11/13  16:53            26,415 sample_submission.csv.zip
2015/08/21  15:08         2,844,086 test.json
2015/11/13  16:53           435,729 test.json.zip
2015/08/21  15:09        12,415,067 train.json
2015/11/13  16:53         1,845,320 train.json.zip
               7 File(s)     17,703,740 bytes
               3 Dir(s)  20,924,313,600 bytes free


In [43]:
train_df = pd.read_json("train.json")
train_df.head()

test_df = pd.read_json("test.json")

In [44]:
train_df['raw_text'] = train_df.ingredients.apply(lambda x: ", ".join(x))
train_df['raw_text'].head()

0    romaine lettuce, black olives, grape tomatoes,...
1    plain flour, ground pepper, salt, tomatoes, gr...
2    eggs, pepper, salt, mayonaise, cooking oil, gr...
3                    water, vegetable oil, wheat, salt
4    black pepper, shallots, cornflour, cayenne pep...
Name: raw_text, dtype: object

In [45]:
train_df.cuisine.head()

0          greek
1    southern_us
2       filipino
3         indian
4         indian
Name: cuisine, dtype: object

In [73]:
train_df['ingredients_clean_text'] = [', '.join(x).strip() for x in train_df['ingredients']]
train_df['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in train_df['ingredients']]

In [66]:
train_df['ingredients_string'].head()

0    romaine lettuce black olives grape tomatoes ga...
1    plain flour ground pepper salt tomato ground b...
2    egg pepper salt mayonaise cooking oil green ch...
3                       water vegetable oil wheat salt
4    black pepper shallot cornflour cayenne pepper ...
Name: ingredients_string, dtype: object

In [74]:
test_df['ingredients_clean_text'] = [', '.join(x).strip() for x in test_df['ingredients']]
test_df['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in test_df['ingredients']]
test_df['ingredients_string'].head()

4987    mushroom chopped onion tomato sauce cheese dri...
9232    minced garlic brown rice sour cream chicken re...
9638    lime juice sesame oil garlic cloves fish sauce...
4927    sugar vanilla extract corn starch coffee granu...
3280    frozen pie crust bourbon whiskey powdered suga...
Name: ingredients_string, dtype: object

In [82]:
corpus_train = train_df['ingredients_string']
vectorizer_train = TfidfVectorizer(stop_words='english')
tfidf_train = vectorizer_train.fit_transform(corpus_train)

In [83]:
corpus_test = test_df['ingredients_string']
vectorizer_test = TfidfVectorizer(stop_words='english')
tfidf_test=vectorizer_train.transform(corpus_test)

In [84]:
predictors_train = tfidf_train
targets_train = train_df['cuisine']

predictors_test = tfidf_test

In [85]:
parameters = {'C':[1, 10]}
# clf = LinearSVC()
clf = LogisticRegression()

classifier = grid_search.GridSearchCV(clf, parameters)
classifier = classifier.fit(predictors_train,targets_train)

In [86]:
classifier

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'C': [1, 10]}, pre_dispatch='2*n_jobs', refit=True,
       score_func=None, scoring=None, verbose=0)

In [87]:
predictions = classifier.predict(predictors_test)
test_df['cuisine'] = predictions
test_df = test_df.sort('id', ascending=True)

In [88]:
test_df.head()

,id,ingredients,ingredients_clean_text,cuisine,ingredients_string
4987,5,"[mushrooms, chopped onion, tomato sauce, chees...","mushrooms, chopped onion, tomato sauce, cheese...",mexican,mushroom chopped onion tomato sauce cheese dri...
9232,7,"[minced garlic, brown rice, sour cream, chicke...","minced garlic, brown rice, sour cream, chicken...",indian,minced garlic brown rice sour cream chicken re...
9638,11,"[lime juice, sesame oil, garlic cloves, fish s...","lime juice, sesame oil, garlic cloves, fish sa...",vietnamese,lime juice sesame oil garlic cloves fish sauce...
4927,12,"[sugar, vanilla extract, corn starch, coffee g...","sugar, vanilla extract, corn starch, coffee gr...",italian,sugar vanilla extract corn starch coffee granu...
3280,13,"[frozen pie crust, bourbon whiskey, powdered s...","frozen pie crust, bourbon whiskey, powdered su...",southern_us,frozen pie crust bourbon whiskey powdered suga...


In [89]:
test_df[['id','cuisine']].to_csv('submission.csv')

In [90]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 000D-0704

 Directory of C:\Users\lv.shuo\Documents\GitHub\Kaggle\Whats cooking

2015/11/13  18:02    <DIR>          .
2015/11/13  18:02    <DIR>          ..
2015/11/13  16:59    <DIR>          .ipynb_checkpoints
2015/11/13  18:02            19,943 ingredients.ipynb
2015/08/26  15:11           137,045 sample_submission.csv
2015/11/13  16:53            26,415 sample_submission.csv.zip
2015/11/13  18:05           200,072 submission.csv
2015/08/21  15:08         2,844,086 test.json
2015/11/13  16:53           435,729 test.json.zip
2015/08/21  15:09        12,415,067 train.json
2015/11/13  16:53         1,845,320 train.json.zip
               8 File(s)     17,923,677 bytes
               3 Dir(s)  20,924,747,776 bytes free
